In [ ]:
import os
import pickle
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score
import cv2

In [ ]:
# importations des modèles 
def load_models():    
    models = []
    with open('models/knn.pkl', 'wb') as f:
        knn = pickle.load(f)
        models.append(knn)
    #with open('models/svm.pkl', 'wb') as f:
    return models

In [ ]:
def create_test_data():
        
    # ================== #
    # Variables globales #
    # ================== #
    if not os.path.exists("dataset/test_data"):
        os.mkdir("test_data")         # création d'un répertoire de test

    nb_images_collectees = 10    # nombre d'image que vous voulez collecter

    donnees_visage = []

    camera = cv2.VideoCapture(0) # 0 pour 'built-in' caméra, 1 pour caméra externe

    cascade_visage = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # ====================================== #
    # Capture des data: images et classes    #
    # ====================================== #

    ret = True
    i = 0

    while(ret):
        ret, frame = camera.read()
        
        if ret == True:
            gris = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            coordonnees_visage = cascade_visage.detectMultiScale(gris, 1.3, 4)

            for (a, b, w, h) in coordonnees_visage:
                features = frame[b:b+h, a:a+w, :]
                visages_redimensionnes = cv2.resize(features, (50, 50))
                
                if i % 10 == 0 and len(donnees_visage) < nb_images_collectees:
                    donnees_visage.append(visages_redimensionnes)
                    
                cv2.rectangle(frame, (a, b), (a+w, b+h), (255, 0, 0), 2)
                
            i += 1

            cv2.imshow('Visages', frame)

            if cv2.waitKey(1) == 27 or len(donnees_visage) >= nb_images_collectees:
                break
        else:
            print('erreur')
            break

    cv2.destroyAllWindows()
    camera.release()

    donnees_visage = np.asarray(donnees_visage)

    # ======================================= #
    # Enregistrement des data: nom et images  #
    # ======================================= #
    if 'test_set.pkl' not in os.listdir('dataset/test_data/'):
        admission_state = 1
        targets = [admission_state] * 10
        with open('dataset/test_data/test_set.pkl', 'wb') as file:
            pickle.dump(targets, file) # Comme le fichier n'existe pas, on part du principe que les visages font partie de la classe admis
    else:
        choix_admission = input("Voulez-vous ajouter ces visages en tant qu'admis? (o/n): ")
        if choix_admission == 'o':
            with open('data_binary_classification/targets.pkl', 'rb') as file:
                targets = pickle.load(file)
            admission_state = 1
            targets = targets + [admission_state] * 10
            with open('data_binary_classification/targets.pkl', 'wb') as file:
                pickle.dump(targets, file)
        else:
            admission_state = 0
            with open('data_binary_classification/targets.pkl', 'rb') as file:
                targets = pickle.load(file)
            targets = targets + [admission_state] * 10
            with open('data_binary_classification/targets.pkl', 'wb') as file:
                pickle.dump(targets, file)

    if 'features.pkl' not in os.listdir('data_binary_classification/'):
        with open('data_binary_classification/features.pkl', 'wb') as w:
            pickle.dump(donnees_visage, w)
    else:
        with open('data_binary_classification/features.pkl', 'rb') as w:
            features = pickle.load(w)
        features = np.append(features, donnees_visage, axis=0)
        with open('data_binary_classification/features.pkl', 'wb') as w:
            pickle.dump(features, w)
        

    cv2.destroyAllWindows()

In [ ]:
models = load_models()
